In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'2.2.0'

In this notebook, we'll convert a keras model into a TF lite model

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [4]:
model = ResNet50(weights='imagenet')

https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg:

<img src="https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg">

In [5]:
url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"

In [6]:
!wget {url}

--2020-06-25 23:33:19--  https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 2620:0:862:ed1a::2:b, 91.198.174.208
Connecting to upload.wikimedia.org (upload.wikimedia.org)|2620:0:862:ed1a::2:b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99389 (97K) [image/jpeg]
Saving to: ‘Pug_600.jpg’

Pug_600.jpg         100%[===================>]  97,06K  --.-KB/s    in 0,08s   

2020-06-25 23:33:20 (1,25 MB/s) - ‘Pug_600.jpg’ saved [99389/99389]



In [7]:
img_path = 'Pug_600.jpg'

In [8]:
img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

In [9]:
decode_predictions(preds, top=3)[0]

[('n02110958', 'pug', 0.99937063),
 ('n02091467', 'Norwegian_elkhound', 0.00053756154),
 ('n02112137', 'chow', 3.7809466e-05)]

Now let's do the same, but without using `load_image` from Keras and loading the image from the internet instead of the disk 

In [10]:
from urllib import request
from io import BytesIO

from PIL import Image

In [11]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

# extracted from 
# https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/image/utils.py

def preprocess_image(img, target_size=(224, 224)):
    # color_mode='rgb',
    # target_size=None,
    # interpolation='nearest',
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def own_preprocess_input(x):
    # caffe preprocessing

    # 'RGB'->'BGR'
    x = x[..., ::-1]

    mean = [103.939, 116.779, 123.68]

    x[..., 0] -= mean[0]
    x[..., 1] -= mean[1]
    x[..., 2] -= mean[2]

    return x

In [12]:
img = download_image(url)
img = preprocess_image(img)

In [13]:
x = np.array(img).astype('float32')
x = np.expand_dims(x, axis=0)
x = own_preprocess_input(x)

preds = model.predict(x)
decode_predictions(preds, top=3)[0]

[('n02110958', 'pug', 0.99937063),
 ('n02091467', 'Norwegian_elkhound', 0.00053756154),
 ('n02112137', 'chow', 3.7809466e-05)]

It works. Now we need to save it to the "saved_model" format, so we can load it later with tf-lite.

In [14]:
output_folder = './tf-model'

In [15]:
tf.saved_model.save(
    model, 
    output_folder,
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./tf-model/assets


Let's load it to make sure it still works in the same way

In [16]:
m = tf.saved_model.load(output_folder)

In [17]:
predict = m.signatures["serving_default"]

In [18]:
preds = predict(tf.constant(x))['predictions'].numpy()

In [19]:
decode_predictions(preds, top=3)[0]

[('n02110958', 'pug', 0.99937063),
 ('n02091467', 'Norwegian_elkhound', 0.00053756154),
 ('n02112137', 'chow', 3.7809466e-05)]

Now let's load from tf-lite and save it as a tf-lite model

In [20]:
converter = tf.lite.TFLiteConverter.from_saved_model(output_folder)

In [21]:
tflite_model = converter.convert()

In [22]:
with tf.io.gfile.GFile('resnet50.tflite', 'wb') as f:
    f.write(tflite_model)

Now let's use a stand-alone tf-lite package for loading this model and doing inference. To use it on ubuntu, get a pre-compiled wheel from https://www.tensorflow.org/lite/guide/python

```
pip install https://dl.google.com/coral/python/tflite_runtime-2.1.0.post1-cp36-cp36m-linux_x86_64.whl
```

(check the link for other python versions)

In [23]:
import tflite_runtime.interpreter as tflite

In [24]:
interpreter = tflite.Interpreter(model_path='resnet50.tflite')
interpreter.allocate_tensors()

In [25]:
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

interpreter.set_tensor(input_index, x)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [27]:
decode_predictions(preds)

[[('n02110958', 'pug', 0.99937063),
  ('n02091467', 'Norwegian_elkhound', 0.0005375595),
  ('n02112137', 'chow', 3.780921e-05),
  ('n02108915', 'French_bulldog', 2.7138167e-05),
  ('n02112350', 'keeshond', 8.389344e-06)]]

Let's re-write `decode_predictions` ourselves to use it in Lambda

- first, get the file with classes, make it more convenient to use and save it in json
- use the clases to decode the predictions

In [28]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json

--2020-06-25 23:39:34--  https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:4001:824::2010, 172.217.22.112, 172.217.23.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:4001:824::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [application/json]
Saving to: ‘imagenet_class_index.json’

imagenet_class_inde 100%[===================>]  34,53K  --.-KB/s    in 0,04s   

2020-06-25 23:39:35 (855 KB/s) - ‘imagenet_class_index.json’ saved [35363/35363]



In [29]:
import json

In [30]:
with open('imagenet_class_index.json') as f_in:
    classes = json.load(f_in)

In [31]:
imagenet_classes = len(classes) * ['NA']

for i, (_, n) in classes.items():
    imagenet_classes[int(i)] = n

In [32]:
with open('imagenet_classes.json', 'w') as f_out:
    json.dump(imagenet_classes, f_out, indent=2)

In [33]:
!head imagenet_classes.json

[
  "tench",
  "goldfish",
  "great_white_shark",
  "tiger_shark",
  "hammerhead",
  "electric_ray",
  "stingray",
  "cock",
  "hen",


In [34]:
with open('imagenet_classes.json') as f_in:
    imagenet_classes = json.load(f_in)

In [35]:
def decode_predictions(preds, top=3):
    results = []

    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]

        result = {imagenet_classes[i]: pred[i] for i in top_indices}
        results.append(result)

    return results

In [36]:
decode_predictions(preds, top=3)[0]

{'pug': 0.99937063, 'Norwegian_elkhound': 0.0005375595, 'chow': 3.780921e-05}

That's all!